<a href="https://colab.research.google.com/github/mion158/data-mining-assignments/blob/main/Recommender_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366462 sha256=36c5783373ad27db599fb4847bc030e086d7f7798e7593534cf8f3b653861b49
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [4]:
import pandas as pd

In [6]:
book = pd.read_csv('https://gist.githubusercontent.com/codecademydev/33dbe210cd84928ff528bfa8af231943/raw/f65d44df8385254f428db651113b97cb560a51f3/goodreads_ratings.csv')
book.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,d089c9b670c0b0b339353aebbace46a1,7686667,3337e0e75701f7f682de11638ccdc60c,3,"Like Matched, this book felt like it was echoi...",Fri Apr 29 14:45:32 -0700 2011,Mon Feb 02 12:57:57 -0800 2015,Sat Jun 18 00:00:00 -0700 2011,Thu May 19 00:00:00 -0700 2011,0,0
1,6dcb2c16e12a41ae0c6c38e9d46f3292,18073066,7201aa3c1161f2bad81258b6d4686c16,5,"WOW again! 4,5 Stars \n So i wont forget to me...",Thu Aug 01 02:15:18 -0700 2013,Mon Nov 18 14:49:26 -0800 2013,Mon Aug 19 00:00:00 -0700 2013,Mon Aug 12 00:00:00 -0700 2013,16,14
2,244e0ce681148a7586d7746676093ce9,13610986,07a203f87bfe1b65ff58774667f6f80d,5,The second novel was hot & heavy. Not only in ...,Sun Nov 23 18:17:50 -0800 2014,Sat May 16 20:34:19 -0700 2015,Fri Dec 19 00:00:00 -0800 2014,Sun Nov 23 00:00:00 -0800 2014,0,0
3,73fcc25ff29f8b73b3a7578aec846394,27274343,8be2d87b07098c16f9742020ec459383,1,What a maddening waste of time. And I unfortun...,Mon Oct 31 08:29:06 -0700 2016,Wed Apr 26 16:06:28 -0700 2017,Wed Apr 26 16:06:28 -0700 2017,Sun Apr 23 00:00:00 -0700 2017,0,1
4,f8880e158a163388a990b64fec7df300,11614718,a29c4ba03e33ad073a414ac775266c5f,4,4.5 stars! \n This was an awesome read! \n So ...,Tue Mar 26 10:55:30 -0700 2013,Mon Sep 08 09:57:05 -0700 2014,Sun Apr 20 09:26:41 -0700 2014,Fri Apr 18 00:00:00 -0700 2014,0,0


In [8]:
#explore ratings
book['rating'].value_counts()

4    1278
5    1001
3     707
2     269
1     125
0     120
Name: rating, dtype: int64

In [10]:
#filter 0 values
book = book[book['rating'] != 0]
book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3380 entries, 0 to 3499
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       3380 non-null   object
 1   book_id       3380 non-null   int64 
 2   review_id     3380 non-null   object
 3   rating        3380 non-null   int64 
 4   review_text   3380 non-null   object
 5   date_added    3380 non-null   object
 6   date_updated  3380 non-null   object
 7   read_at       3116 non-null   object
 8   started_at    2347 non-null   object
 9   n_votes       3380 non-null   int64 
 10  n_comments    3380 non-null   int64 
dtypes: int64(4), object(7)
memory usage: 316.9+ KB


In [14]:
#build surprise reader object
from surprise import Reader
reader = Reader(rating_scale=(1,5))

#load book into surprise dataset
from surprise import Dataset
book_data = Dataset.load_from_df(book[['user_id','book_id','rating']], reader)

#split dataset
from surprise.model_selection import train_test_split
train_set, test_set = train_test_split(book_data, test_size=0.2, random_state=7)

In [15]:
#train collab filtering
from surprise import KNNBasic
nn_algo = KNNBasic()
nn_algo.fit(train_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [16]:
#evaluate
from surprise import accuracy
predictions = nn_algo.test(test_set)
accuracy.rmse(predictions)

RMSE: 1.1105


1.110471008157185

In [17]:
#test recommendation
nn_algo.predict('8842281e1d1347389f2ab93d60773d4d', '18007564').est

3.8250739644970415